## チェックポイントでモデル構造の復元を試す
- Reference
  - https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja
  - https://www.tensorflow.org/guide/checkpoint

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.2.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

print(train_images.shape)

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

print(train_images.shape)

(60000, 28, 28)
(1000, 784)


### モデルを作成して学習させつつ CheckpointManager で保存する

In [3]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint_path = 'data/checkpoint_architecture/cp.ckpt'

ckpt = tf.train.Checkpoint(model_1=model)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [8]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs = 10, 
          validation_data = (test_images, test_labels),
          verbose=0)

In [9]:
print(ckpt_manager.latest_checkpoint)

None


In [10]:
ckpt_manager.save()

print(ckpt_manager.latest_checkpoint)

data/checkpoint_architecture/cp.ckpt/ckpt-1


### ダミーのモデルを作成して CheckpointManager から復元する

In [19]:
def create_model_dummy():
  model = tf.keras.models.Sequential()

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


model2 = create_model_dummy()

In [20]:
ckpt2 = tf.train.Checkpoint(model_1=model2)

ckpt_manager2 = tf.train.CheckpointManager(ckpt2, checkpoint_path, max_to_keep=5)

In [21]:
print(ckpt_manager2.latest_checkpoint)

data/checkpoint_architecture/cp.ckpt/ckpt-1


In [22]:
ckpt2.restore(ckpt_manager2.latest_checkpoint)

In [23]:
model2.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.